In [2]:
import numpy as np
import pandas as pd

## Datasets Used

In the `data` folder, we include a `data_sources.tsv` to keep track of the datasets we find useful. For each dataset, we record the granularity, url, and the date we retrieved the data. If the datasets were published on Github, we also record the url of the specific version.


We kept the original name of the datasets, and only performed necessary actions:

- All dataset from World Bank have metadata at the top (line 1-4). We removed those lines.
- `kff/raw_data.csv` includes some extra information in the spreadsheet (line 1-2, 56-71), which cause errors. We removed those lines from the table.

In [3]:
used_data = pd.read_csv('./data/data_sources.tsv', sep='\t')
used_data[['name', 'url', 'date_retrieved']]

,name,url,date_retrieved
0,covid-tracking-states-current,https://github.com/COVID19Tracking/covid-track...,2020-04-23
1,covid-tracking-states-daily,https://github.com/COVID19Tracking/covid-track...,2020-04-23
2,covid-tracking-us-current,https://github.com/COVID19Tracking/covid-track...,2020-04-23
3,covid-tracking-us-daily,https://github.com/COVID19Tracking/covid-track...,2020-04-23
4,khn-icu-beds-by-county,https://khn.org/news/as-coronavirus-spreads-wi...,2020-04-21
5,kff-state-actions,https://www.kff.org/health-costs/issue-brief/s...,2020-04-24
6,world-bank-hospital-beds,https://data.worldbank.org/indicator/SH.MED.BE...,2020-04-10
7,world-bank-physicians,https://data.worldbank.org/indicator/SH.MED.PH...,2020-04-24
8,world-bank-nurses,https://data.worldbank.org/indicator/SH.MED.NU...,2020-04-24
9,world-bank-specialist-surgical-workforce,https://data.worldbank.org/indicator/SH.MED.SA...,2020-04-24
